In [1]:
import os
import pandas as pd
pd.options.display.max_colwidth = 160

import preprocessing as util
from raw_utils import save_to_csv

In [2]:
# Path
cwd = os.getcwd()
csv_path = os.path.join(cwd, 'data/csv/')

data_files = ['balanced.csv', 'imbalanced.csv']

In [3]:
balanced = pd.read_csv(os.path.join(csv_path, data_files[0]), index_col=0, dtype={'body': 'object', 'class': 'bool', 'id': 'int16'})
balanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3376 entries, 0 to 3375
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      3376 non-null   int16 
 1   body    3376 non-null   object
 2   class   3376 non-null   bool  
dtypes: bool(1), int16(1), object(1)
memory usage: 62.6+ KB


In [4]:
imbalanced = pd.read_csv(os.path.join(csv_path, data_files[1]), index_col=0, dtype={'body': 'object', 'class': 'bool', 'id': 'int16'})
imbalanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18568 entries, 0 to 18567
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18568 non-null  int16 
 1   body    18568 non-null  object
 2   class   18568 non-null  bool  
dtypes: bool(1), int16(1), object(1)
memory usage: 344.5+ KB


### Initial Data

This is the initial state of the data, along with some representative examples of phishing and legitimate emails.

In [5]:
balanced.head(20)

,id,body,class
0,0,Dear jose\nYour E-mail jose@monkey.org would be blocked from sending and receiving emails.if you have not update your account in our New 2020 security datab...,True
1,1,Enron Still Target in California Amid Other Problems (Update1)\n2001-10-26 16:15 (New York)\n\nEnron Still Target in California Amid Other Problems (Update1...,False
2,2,"PLEASE SEND ME YOUR PHONE NUMBER!. We need to set up a short conference call \nto understand the counterparty. Thanks. Sara\n\n\n\n\t""Susan Hopkinson"" <s...",False
3,3,"Stinson-san & Co,\n\nWe may become famous in Japan. Please, see the message below.\n\nVince\n\n---------------------- Forwarded by Vince J Kaminski/HOU/ECT ...",False
4,4,"In English this means that even if a book has no positions or value, if it \nhas any deals with a counterparty it needs to be officialized so that credit \...",False
5,5,"Jim:\n\nI agree. However, it probably would make some sense to have EWS legal look \nat any particularly complex pricing structures that may be passed by t...",False
6,6,"monkey.org\nHi jose,\nOur server security is holding (5) incoming messages because your email is not yet validate to our newest server upgrade .\nIf you wi...",True
7,7,CALENDAR ENTRY:\tAPPOINTMENT\n\nDescription:\n\tInterview Z\n\nDate:\t\t12/14/2000\nTime:\t\t9:00 AM - 9:30 AM (Central Standard Time)\n\nChairperson:\tOutl...,False
8,8,"This is to inform you that your password will expire in 3 days, please update your account or your new mails will remain pending.\n\nNote: Open http://www.p...",True
9,9,I never received this. Is it mine?\n\n\n\n\tBecky Stephens@ENRON\n\t12/11/2000 04:07 PM\n\t\t \n\t\t To: richard.sanders@enron.com\n\t\t cc: linda.guinn@enr...,False


#### Legitimate emails:

In [6]:
print(balanced['body'].iloc[1])

Enron Still Target in California Amid Other Problems (Update1)
2001-10-26 16:15 (New York)

Enron Still Target in California Amid Other Problems (Update1)

     (Updates with closing share price in last paragraph.)

     Sacramento, California, Oct. 26 (Bloomberg) -- Enron Corp.,
facing an inquiry by federal securities regulators into
partnerships run by the former chief financial officer, remains a
target of investigations and lawsuits in California.

     California lawmakers and regulators have accused power
providers of manipulating the state's energy market to raise
prices. Enron, the biggest energy trader, and other power sellers
have denied the charges repeatedly.

     Next month, a California Senate committee investigating the
power market plans to hold a hearing to determine if Enron and
other generators are complying with subpoenas for trading
documents. Enron has been filing documents in Sacramento,
California.

     ``They are still putting documents in their depository, a

In [7]:
print(balanced['body'].iloc[16])

I will be running the power options desk with UBS. I will still be trading gas though. how about you ?



In [8]:
print(balanced['body'].iloc[18])

The only documents are the two redlines from Florio and Edison - and you
were on that distribution list -- did you not receive them? Let me know.

-----Original Message-----
From: Jeff.Dasovich@enron.com [mailto:Jeff.Dasovich@enron.com]
Sent: Wednesday, June 13, 2001 11:06 AM
To: dhunter@s-k-w.com
Subject: Re: Meeting Time and Final Letter



Can I get a copy of the full document?  Is it ready yet?

Thanks a million.

Best,
Jeff



#### Phishing Emails:

In [9]:
print(balanced['body'].iloc[6])

monkey.org
Hi  jose,
Our server security is holding (5) incoming messages because your email is not yet validate to our newest server upgrade .
If you wish to keep using our service, kindly upgarde your email by 
Signing in here<http://jacksonuy.tk/hme/enx.php?email=jose@monkey.org> .
You will not be able to access your email if Ignored, upgrade takes few seconds .
Thank you,
monkey.org Team
All other trademarks are the property of their respective owners.  © 2017 Mail Systems Incorporated. All rights reserved.


In [10]:
print(balanced['body'].iloc[11])

Dear Customer,

For your safety, We are pleased to attach your new message notification in PDF File. 

Kindly open the attached PDF file to your email to read your message notification.

Thank you, 

USAA


In [11]:
print(balanced['body'].iloc[17])

 
 

  
    	  
   
 
   
 
   Server Message
  

 
   Dear jose@monkey.org , Our record indicates that you recently made a request to shutdown your e-mail and this request will be processed shortly.
 If this request was made accidentally and you have no knowledge of it, you are advised to cancel the request now

   
   
    Cancel De-activation

   
   
   However, if you do not cancel this request your account will be shutdown shortly and all your email data will be lost permanently. 
 Regards. 
Email Administrator

   
   
    
 
   
   This message is auto-generated from E-mail security server, and replies sent to this email can not be delivered. This email is meant for: Account User 

 
 

Server Message
Dear
jose@monkey.org
<mailto:25> ,
Our record indicates that you recently made a request to shutdown your e-mail and this request will be processed shortly.
If this request was made accidentally and you have no knowledge of it, you are advised to cancel the request now
Cancel De-a

In [12]:
print(balanced['body'].iloc[3240])

Your e-mailbox password will expire in 2 days. to keep your password. CLICK=HERE<http://owaogr.tripod.com/> to update immediately

IT-Service Help Desk.



Some observations:
- There is a lot of extra whitespace that can be sanitized later.
- There still exist some emails with duplicated text, mostly because of the way the links from \<a\> tags were extracted.
- Emails and URLs can give away the class of the message (domain enron.com vs domain monkey.org), so removing them should make the model more general.

# Preprocessing

We need to convert the text data into a format more suitable for use with machine learning algorithms.<br>
Since we aim for two different feature sets, the process will be split.

## Basic Preprocessing

These processeses should happen to all the feature sets.

### Replacing addresses

As is obvious from the examples, a lot of the emails contain either **web addresses** (URLs) or **email addresses** that need to be removed in order for the frequency of certain domains to not influence the results.<br>
In order for this information to not get completely lost however, those addresses will be replaced by the strings `'<urladdress>'` and `'<emailaddress>'` respectively. Those strings are chosen because they do not occur normally in the emails.

In [13]:
balanced['body'] = balanced['body'].apply(util.replace_email)
balanced['body'] = balanced['body'].apply(util.replace_url)

In [14]:
imbalanced['body'] = imbalanced['body'].apply(util.replace_email)
imbalanced['body'] = imbalanced['body'].apply(util.replace_url)

In [15]:
print(balanced['body'].iloc[6])

<urladdress>
Hi  jose,
Our server security is holding (5) incoming messages because your email is not yet validate to our newest server upgrade .
If you wish to keep using our service, kindly upgarde your email by 
Signing in here<<urladdress><emailaddress>> .
You will not be able to access your email if Ignored, upgrade takes few seconds .
Thank you,
<urladdress> Team
All other trademarks are the property of their respective owners.  © 2017 Mail Systems Incorporated. All rights reserved.


In [16]:
print(balanced['body'].iloc[17])

 
 

  
    	  
   
 
   
 
   Server Message
  

 
   Dear <emailaddress> , Our record indicates that you recently made a request to shutdown your e-mail and this request will be processed shortly.
 If this request was made accidentally and you have no knowledge of it, you are advised to cancel the request now

   
   
    Cancel De-activation

   
   
   However, if you do not cancel this request your account will be shutdown shortly and all your email data will be lost permanently. 
 Regards. 
Email Administrator

   
   
    
 
   
   This message is auto-generated from E-mail security server, and replies sent to this email can not be delivered. This email is meant for: Account User 

 
 

Server Message
Dear
<emailaddress>
<mailto:25> ,
Our record indicates that you recently made a request to shutdown your e-mail and this request will be processed shortly.
If this request was made accidentally and you have no knowledge of it, you are advised to cancel the request now
Cancel De-act

The examples show that the URLs and email addresses have indeed been anonymized now.

## Preprocessing for content features

This preprocessing is necessary in order to convert the text strings to lists of words, that will be vectorized in order to be used by machine learning algorithms.

In [17]:
balanced_tokens = balanced.copy()
imbalanced_tokens = imbalanced.copy()

### Tokenization and stopword removal

Tokenization is the process of splitting text into individual words. This is useful because generally speaking, the meaning of the text can easily be interpreted by analyzing the words present in the text.<br>
Along with this process, letters are also converted to lowercase and punctuation or other special characters are removed.<br>
Since there are some words (called **stopwords**) that do not contribute very much in meaning (like pronouns or simple verbs), they can be removed to reduce the noise.

In [18]:
balanced_tokens['body'] = balanced_tokens['body'].apply(util.tokenize)
balanced_tokens['body'] = balanced_tokens['body'].apply(util.remove_stopwords)

In [19]:
imbalanced_tokens['body'] = imbalanced_tokens['body'].apply(util.tokenize)
imbalanced_tokens['body'] = imbalanced_tokens['body'].apply(util.remove_stopwords)

In [20]:
print(balanced_tokens['body'].iloc[6])

['urladdress', 'hi', 'jose', 'server', 'security', 'holding', '5', 'incoming', 'messages', 'email', 'yet', 'validate', 'newest', 'server', 'upgrade', 'wish', 'keep', 'using', 'service', 'kindly', 'upgarde', 'email', 'signing', 'urladdress', 'emailaddress', 'able', 'access', 'email', 'ignored', 'upgrade', 'takes', 'seconds', 'thank', 'urladdress', 'team', 'trademarks', 'property', 'respective', 'owners', '2017', 'mail', 'systems', 'incorporated', 'rights', 'reserved']


In [21]:
print(balanced_tokens['body'].iloc[17])

['server', 'message', 'dear', 'emailaddress', 'record', 'indicates', 'recently', 'made', 'request', 'shutdown', 'e-mail', 'request', 'processed', 'shortly', 'request', 'made', 'accidentally', 'knowledge', 'advised', 'cancel', 'request', 'cancel', 'de-activation', 'however', 'cancel', 'request', 'account', 'shutdown', 'shortly', 'email', 'data', 'lost', 'permanently', 'regards', 'email', 'administrator', 'message', 'auto-generated', 'e-mail', 'security', 'server', 'replies', 'sent', 'email', 'delivered', 'email', 'meant', 'account', 'user', 'server', 'message', 'dear', 'emailaddress', 'record', 'indicates', 'recently', 'made', 'request', 'shutdown', 'e-mail', 'request', 'processed', 'shortly', 'request', 'made', 'accidentally', 'knowledge', 'advised', 'cancel', 'request', 'cancel', 'de-activation', 'urladdress', 'emailaddress', 'however', 'cancel', 'request', 'account', 'shutdown', 'shortly', 'email', 'data', 'lost', 'permanently', 'regards', 'email', 'administrator', 'message', 'auto-g

The example shows how a quite big chunk of text was reduced to a smaller list that contains the more meaningful words. The addresses still exist as tokens ('urladdress').<br>
Also, some emails with duplicate emails obviously will have duplicate tokens, this however is not a big issue with most vectorizers.

### Lemmatization with POS tagging

Lemmatization is the process that reduces the inflectional forms of a word to keep its root form. This is useful because the set of words that results from this process is smaller because all the inflections of a word are converted to one, thus reducing the dimensionality without sacrificing information.<br>
In order to facilitate and improve the lemmatization, the **part-of-speech tagging** technique has been used. The POS of the word (which indicates whether a word is a noun, a verb, an adjective, or an adverb) is used as a part of the process.

In [22]:
balanced_tokens['body'] = balanced_tokens['body'].apply(util.lemmatize)

In [23]:
imbalanced_tokens['body'] = imbalanced_tokens['body'].apply(util.lemmatize)

In [24]:
print(balanced_tokens['body'].iloc[6])

['urladdress', 'hi', 'jose', 'server', 'security', 'hold', '5', 'incoming', 'message', 'email', 'yet', 'validate', 'new', 'server', 'upgrade', 'wish', 'keep', 'use', 'service', 'kindly', 'upgarde', 'email', 'signing', 'urladdress', 'emailaddress', 'able', 'access', 'email', 'ignore', 'upgrade', 'take', 'second', 'thank', 'urladdress', 'team', 'trademark', 'property', 'respective', 'owner', '2017', 'mail', 'system', 'incorporate', 'right', 'reserve']


The example shows how the lemmatization process has worked: words like 'holding' have been converted to their root form 'hold'.<br>
In addition, it also shows the working of the POS tagging process, since the word 'incoming' has remained the same as it is used as an adjective and not as a verb.

## Preprocessing for style features

This preprocessing is necessary in order to sanitize the raw email text and remove parsing artifacts, so that the stylometric features work better.

In [25]:
balanced_text = balanced.copy()
imbalanced_text = imbalanced.copy()

### Whitespace Sanitization

The first task should be stripping away any leading and trailing whitespace. In addition, newlines that only contain a dot are most likely artifacts from the parsing of HTML and can thus be removed with this dot placed at the previous line.

In [26]:
balanced_text['body'] = balanced_text['body'].apply(util.sanitize_whitespace)

In [27]:
imbalanced_text['body'] = imbalanced_text['body'].apply(util.sanitize_whitespace)

### Address Sanitization

There are also artifacts from the URL/email anonymization that while innocent with tokenized texts, they are more harmful when the number of special characters in the text matters.

In [28]:
balanced_text['body'] = balanced_text['body'].apply(util.sanitize_addresses)

In [29]:
imbalanced_text['body'] = imbalanced_text['body'].apply(util.sanitize_addresses)

One of the previous examples looks better now.

In [30]:
print(balanced_text['body'].iloc[6])

<urladdress>
Hi  jose,
Our server security is holding (5) incoming messages because your email is not yet validate to our newest server upgrade .
If you wish to keep using our service, kindly upgarde your email by 
Signing in here<urladdress><emailaddress> .
You will not be able to access your email if Ignored, upgrade takes few seconds .
Thank you,
<urladdress> Team
All other trademarks are the property of their respective owners.  © 2017 Mail Systems Incorporated. All rights reserved.


## Deleting Empty Rows

After all the preprocessing, it is possible that some of the emails are now empty (because they did not contain any useful words from the beginning).<br>
So, these have to be removed to keep the data clean.

In [31]:
balanced_tokens = balanced_tokens[balanced_tokens['body'].astype(bool)]
balanced_tokens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3368 entries, 0 to 3375
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      3368 non-null   int16 
 1   body    3368 non-null   object
 2   class   3368 non-null   bool  
dtypes: bool(1), int16(1), object(1)
memory usage: 62.5+ KB


In [32]:
imbalanced_tokens = imbalanced_tokens[imbalanced_tokens['body'].astype(bool)]
imbalanced_tokens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18498 entries, 0 to 18567
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18498 non-null  int16 
 1   body    18498 non-null  object
 2   class   18498 non-null  bool  
dtypes: bool(1), int16(1), object(1)
memory usage: 343.2+ KB


In [33]:
balanced_text = balanced_text[balanced_text['body'].astype(bool)]
balanced_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3375 entries, 0 to 3375
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      3375 non-null   int16 
 1   body    3375 non-null   object
 2   class   3375 non-null   bool  
dtypes: bool(1), int16(1), object(1)
memory usage: 62.6+ KB


In [34]:
imbalanced_text = imbalanced_text[imbalanced_text['body'].astype(bool)]
imbalanced_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18567 entries, 0 to 18567
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18567 non-null  int16 
 1   body    18567 non-null  object
 2   class   18567 non-null  bool  
dtypes: bool(1), int16(1), object(1)
memory usage: 344.5+ KB


In order to have the same emails in both feature sets, the text dataset will be filtered according to the tokenized one.

In [35]:
balanced_text = balanced_text[balanced_text['id'].isin(balanced_tokens['id'])]
balanced_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3368 entries, 0 to 3375
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      3368 non-null   int16 
 1   body    3368 non-null   object
 2   class   3368 non-null   bool  
dtypes: bool(1), int16(1), object(1)
memory usage: 62.5+ KB


In [36]:
imbalanced_text = imbalanced_text[imbalanced_text['id'].isin(imbalanced_tokens['id'])]
imbalanced_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18498 entries, 0 to 18567
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18498 non-null  int16 
 1   body    18498 non-null  object
 2   class   18498 non-null  bool  
dtypes: bool(1), int16(1), object(1)
memory usage: 343.2+ KB


Check for any discrepancies:

In [37]:
(balanced_text['id'] != balanced_tokens['id']).any() and (imbalanced_text['id'] != imbalanced_tokens['id']).any()

False

## Train-Test Split

In order to evaluate the classification process, only 80% of the data will be used to train the models. The remaining 20%, which will be unknown to the algorithms, will be used to test the performance of the classifiers on unknown data.

### Tokens

In [38]:
train_balanced_tokens, test_balanced_tokens = util.dataset_split(balanced_tokens, percent=20)

In [39]:
train_imbalanced_tokens, test_imbalanced_tokens = util.dataset_split(imbalanced_tokens, percent=20)

In [40]:
train_balanced_tokens[train_balanced_tokens['id'] == 6]

,id,body,class
2089,6,"[urladdress, hi, jose, server, security, hold, 5, incoming, message, email, yet, validate, new, server, upgrade, wish, keep, use, service, kindly, upgarde, ...",True


In [41]:
test_balanced_tokens[test_balanced_tokens['id'] == 17]

,id,body,class
278,17,"[server, message, dear, emailaddress, record, indicate, recently, make, request, shutdown, e-mail, request, process, shortly, request, make, accidentally, k...",True


One of the examples is on the train set while the other is on the test set.

### Text

In [42]:
train_balanced_text, test_balanced_text = util.dataset_split(balanced_text, percent=20)

In [43]:
train_imbalanced_text, test_imbalanced_text = util.dataset_split(imbalanced_text, percent=20)

Confirm that the train and test datasets do not have any different emails:

In [44]:
(train_balanced_text['id'] != train_balanced_tokens['id']).any() and (train_imbalanced_text['id'] != train_imbalanced_tokens['id']).any() and (test_balanced_text['id'] != test_balanced_tokens['id']).any() and (test_imbalanced_text['id'] != test_imbalanced_tokens['id']).any()

False

### Saving the Results

#### Tokens

In [45]:
save_to_csv(train_balanced_tokens, csv_path, 'train_balanced_tokens.csv')
save_to_csv(test_balanced_tokens, csv_path, 'test_balanced_tokens.csv')

Saving to /home/ichanis/projects/phishing_public/data/csv/train_balanced_tokens.csv
Saving to /home/ichanis/projects/phishing_public/data/csv/test_balanced_tokens.csv


In [46]:
save_to_csv(train_imbalanced_tokens, csv_path, 'train_imbalanced_tokens.csv')
save_to_csv(test_imbalanced_tokens, csv_path, 'test_imbalanced_tokens.csv')

Saving to /home/ichanis/projects/phishing_public/data/csv/train_imbalanced_tokens.csv
Saving to /home/ichanis/projects/phishing_public/data/csv/test_imbalanced_tokens.csv


#### Text

In [47]:
save_to_csv(train_balanced_text, csv_path, 'train_balanced_text.csv')
save_to_csv(test_balanced_text, csv_path, 'test_balanced_text.csv')

Saving to /home/ichanis/projects/phishing_public/data/csv/train_balanced_text.csv
Saving to /home/ichanis/projects/phishing_public/data/csv/test_balanced_text.csv


In [48]:
save_to_csv(train_imbalanced_text, csv_path, 'train_imbalanced_text.csv')
save_to_csv(test_imbalanced_text, csv_path, 'test_imbalanced_text.csv')

Saving to /home/ichanis/projects/phishing_public/data/csv/train_imbalanced_text.csv
Saving to /home/ichanis/projects/phishing_public/data/csv/test_imbalanced_text.csv
